# Data

In [1]:
import torch 
import importlib
import monotonic
import data_manager
import metrics
import utils
importlib.reload(utils)
import train as Train
from train import execute_model_evaluation
import train_config
from data_manager import DatasetManager
from train_config import FlatACLConfig, FlatDLinearConfig, FlatNaiveConfig, FlatPatchTSTConfig, FlatTimeMixerConfig
from dataclasses import replace

%load_ext autoreload
%autoreload 2
modules_to_reload_list = [
    data_manager,
    Train,
    train_config,
    monotonic,
    # data_manager, # Reloaded only once even if listed twice
    utils,
    # train_config, # Reloaded only once even if listed twice
    metrics
]

# Initialize the data manager
data_mgr = DatasetManager(device='cuda')

# Load a synthetic dataset
data_mgr.load_trajectory('rossler', steps=51999, dt=1e-2) # 50399

RosslerSystem initialized with method: rk4 on device: cuda

Dataset: rossler (synthetic)
Shape: torch.Size([52000, 3])
Channels: 3
Length: 52000
Parameters: {'steps': 51999, 'dt': 0.01}

Sample data (first 2 rows):
tensor([[1.0000, 1.0000, 1.0000],
        [0.9802, 1.0119, 0.9559]], device='cuda:0')


# Rossler

In [ ]:
data_mgr.datasets

## Seq=96

### EigenACL

#### pred=96

##### huber 

In [12]:
from monotonic import DynamicTanh
import torch.nn as nn

importlib.reload(monotonic)
importlib.reload(train_config) 
cfg = train_config.FlatACLConfig(  # original householder 
    seq_len=96,
    pred_len=96,
    channels=data_mgr.datasets['rossler']['channels'],# data_mgr.channels,              # ← number of features in your data
    batch_size=128,
    learning_rate=9e-4, 
    seeds=[1955, 7, 20],  
    epochs=50, 
    dim_hidden=128,
    dim_augment=128, 
    ablate_no_koopman=False,
    use_complex_eigenvalues=True,
    second_delay_use_shift=True,
    ablate_rotate_back_Koopman=True, 
    ablate_shift_inside_scale=False,
    householder_reflects_latent = 2,
    householder_reflects_data = 4,
    mixing_strategy='delay_only', 
    loss_backward_weights = [0.0, 0.0, 1.0, 0.0, 0.0],
    loss_validate_weights = [0.0, 0.0, 1.0, 0.0, 0.0],
    ablate_deterministic_y0=False, 
)
cfg.x_to_z_delay.enable_magnitudes = [False, True]
cfg.x_to_z_delay.spectral_flags_scale_shift = [True, False]
cfg.x_to_z_delay.spectral_flags_magnitudes = [False, True]
cfg.x_to_z_delay.spectral_flags_hidden_layers = [False, False]
cfg.x_to_z_delay.activations_scale_shift = ["relu6", "dynamic_tanh"]
cfg.x_to_z_delay.activations_hidden_layers = [nn.ELU, nn.LogSigmoid]

cfg.x_to_z_deri.enable_magnitudes = [False, True]
cfg.x_to_z_deri.spectral_flags_scale_shift = [True, False]
cfg.x_to_z_deri.spectral_flags_magnitudes = [False, True]
cfg.x_to_z_deri.spectral_flags_hidden_layers = [False, False]
cfg.x_to_z_deri.activations_scale_shift = ["relu6", "dynamic_tanh"]
cfg.x_to_z_deri.activations_hidden_layers = [nn.ELU, nn.LogSigmoid]

cfg.z_to_x_main.enable_magnitudes = [False, True]
cfg.z_to_x_main.spectral_flags_scale_shift = [True, False]
cfg.z_to_x_main.spectral_flags_magnitudes = [False, True]
cfg.z_to_x_main.spectral_flags_hidden_layers = [False, False]
cfg.z_to_x_main.activations_scale_shift = ["relu6", "dynamic_tanh"]
cfg.z_to_x_main.activations_hidden_layers = [nn.ELU, nn.LogSigmoid]

cfg.z_push_to_z.enable_magnitudes = [False, True]
cfg.z_push_to_z.spectral_flags_scale_shift = [True, False]
cfg.z_push_to_z.spectral_flags_magnitudes = [False, True]
cfg.z_push_to_z.spectral_flags_hidden_layers = [False, False]
cfg.z_push_to_z.activations_scale_shift = ["relu6", "dynamic_tanh"]
cfg.z_push_to_z.activations_hidden_layers = [nn.ELU, nn.LogSigmoid]

cfg.z_to_y_main.enable_magnitudes = [False, True]
cfg.z_to_y_main.spectral_flags_scale_shift = [True, False]
cfg.z_to_y_main.spectral_flags_magnitudes = [False, True]
cfg.z_to_y_main.spectral_flags_hidden_layers = [False, False]
cfg.z_to_y_main.activations_scale_shift = ["relu6", "dynamic_tanh"]
cfg.z_to_y_main.activations_hidden_layers = [nn.ELU, nn.LogSigmoid]

# cfg.x_to_z_delay.scale_zeroing_threshold = 1e-4
# cfg.x_to_z_deri.scale_zeroing_threshold = 1e-4
# cfg.z_to_x_main.scale_zeroing_threshold = 1e-4
# cfg.z_push_to_z.scale_zeroing_threshold = 1e-4
# cfg.z_to_y_main.scale_zeroing_threshold = 1e-4
exp = execute_model_evaluation('rossler', cfg, data_mgr, scale=False)

Shape of training data: torch.Size([36400, 3])
Shape of validation data: torch.Size([5200, 3])
Shape of testing data: torch.Size([10400, 3])
global_std.shape: torch.Size([3])
Global Std for rossler: tensor([5.0964, 4.7939, 2.8266], device='cuda:0')
Train set sample shapes: torch.Size([96, 3]), torch.Size([96, 3])
Validation set sample shapes: torch.Size([96, 3]), torch.Size([96, 3])
Test set data shapes: torch.Size([10400, 3]), torch.Size([10400, 3])
Number of batches in train_loader: 283
Batch 0: Data shape torch.Size([128, 96, 3]), Target shape torch.Size([128, 96, 3])

Data Preparation: rossler
Sequence Length: 96
Prediction Length: 96
Batch Size: 128
Scaling: No
Train Split: 0.7
Val Split: 0.8
Training Batches: 283
Validation Batches: 40
Test Batches: 80

 Running experiment with seed 1955 (1/3)==================================================

Epoch [1/50], Train Losses: mse: 2.3434, mae: 0.5442, huber: 0.3462, swd: 1.9665, ept: 91.2373
Epoch [1/50], Val Losses: mse: 0.8194, mae:

##### do not use ablations: rotations (4,2)

In [6]:
importlib.reload(monotonic)
importlib.reload(train_config)

cfg = train_config.FlatACLConfig( 
    seq_len=96,
    pred_len=96,
    channels=data_mgr.datasets['rossler']['channels'],# data_mgr.channels,              # ← number of features in your data
    batch_size=128,
    learning_rate=9e-4, 
    seeds=[1955, 7, 20],  
    epochs=50, 
    dim_hidden=128,
    dim_augment=128,
    ablate_no_koopman=False,
    use_complex_eigenvalues=True,
    second_delay_use_shift=True,
    ablate_rotate_back_Koopman=False, 
    ablate_shift_inside_scale=False,
)
cfg.x_to_z_delay.enable_magnitudes_scale_shift = [False, True]
cfg.x_to_z_deri.enable_magnitudes_scale_shift = [False, True]
cfg.z_to_x_main.enable_magnitudes_scale_shift = [False, True]
cfg.z_to_y_main.enable_magnitudes_scale_shift = [False, True]
exp = execute_model_evaluation('rossler', cfg, data_mgr, scale=False)

Shape of training data: torch.Size([36400, 3])
Shape of validation data: torch.Size([5200, 3])
Shape of testing data: torch.Size([10400, 3])
Train set sample shapes: torch.Size([96, 3]), torch.Size([96, 3])
Validation set sample shapes: torch.Size([96, 3]), torch.Size([96, 3])
Test set data shapes: torch.Size([10400, 3]), torch.Size([10400, 3])
Number of batches in train_loader: 283
Batch 0: Data shape torch.Size([128, 96, 3]), Target shape torch.Size([128, 96, 3])

Data Preparation: rossler
Sequence Length: 96
Prediction Length: 96
Batch Size: 128
Scaling: No
Train Split: 0.7
Val Split: 0.8
Training Batches: 283
Validation Batches: 40
Test Batches: 80

 Running experiment with seed 1955 (1/3)==================================================

Epoch [1/50], Train Losses: mse: 2.3266, mae: 0.5618, huber: 0.3521, swd: 1.9839, target_std: 3.9251
Epoch [1/50], Val Losses: mse: 0.5240, mae: 0.2358, huber: 0.0785, swd: 0.4133, target_std: 3.9370
Epoch [1/50], Test Losses: mse: 0.4725, mae: 0

##### do not use ablations: Threshold: all 

In [13]:
from monotonic import DynamicTanh
import torch.nn as nn

importlib.reload(monotonic)
importlib.reload(train_config) 
cfg = train_config.FlatACLConfig(  # original householder 
    seq_len=96,
    pred_len=96,
    channels=data_mgr.datasets['rossler']['channels'],# data_mgr.channels,              # ← number of features in your data
    batch_size=128,
    learning_rate=9e-4, 
    seeds=[1955, 7, 20],  
    epochs=50, 
    dim_hidden=128,
    dim_augment=128, 
    ablate_no_koopman=False,
    use_complex_eigenvalues=True,
    second_delay_use_shift=True,
    ablate_rotate_back_Koopman=True, 
    ablate_shift_inside_scale=False,
    householder_reflects_latent = 2,
    householder_reflects_data = 4,
    mixing_strategy='delay_only', 
    loss_backward_weights = [0.0, 0.0, 1.0, 0.0, 0.0],
    loss_validate_weights = [0.0, 0.0, 1.0, 0.0, 0.0],
    ablate_deterministic_y0=False, 
)
cfg.x_to_z_delay.enable_magnitudes = [False, True]
cfg.x_to_z_delay.spectral_flags_scale_shift = [True, False]
cfg.x_to_z_delay.spectral_flags_magnitudes = [False, True]
cfg.x_to_z_delay.spectral_flags_hidden_layers = [False, False]
cfg.x_to_z_delay.activations_scale_shift = ["relu6", "dynamic_tanh"]
cfg.x_to_z_delay.activations_hidden_layers = [nn.ELU, nn.LogSigmoid]

cfg.x_to_z_deri.enable_magnitudes = [False, True]
cfg.x_to_z_deri.spectral_flags_scale_shift = [True, False]
cfg.x_to_z_deri.spectral_flags_magnitudes = [False, True]
cfg.x_to_z_deri.spectral_flags_hidden_layers = [False, False]
cfg.x_to_z_deri.activations_scale_shift = ["relu6", "dynamic_tanh"]
cfg.x_to_z_deri.activations_hidden_layers = [nn.ELU, nn.LogSigmoid]

cfg.z_to_x_main.enable_magnitudes = [False, True]
cfg.z_to_x_main.spectral_flags_scale_shift = [True, False]
cfg.z_to_x_main.spectral_flags_magnitudes = [False, True]
cfg.z_to_x_main.spectral_flags_hidden_layers = [False, False]
cfg.z_to_x_main.activations_scale_shift = ["relu6", "dynamic_tanh"]
cfg.z_to_x_main.activations_hidden_layers = [nn.ELU, nn.LogSigmoid]

cfg.z_push_to_z.enable_magnitudes = [False, True]
cfg.z_push_to_z.spectral_flags_scale_shift = [True, False]
cfg.z_push_to_z.spectral_flags_magnitudes = [False, True]
cfg.z_push_to_z.spectral_flags_hidden_layers = [False, False]
cfg.z_push_to_z.activations_scale_shift = ["relu6", "dynamic_tanh"]
cfg.z_push_to_z.activations_hidden_layers = [nn.ELU, nn.LogSigmoid]

cfg.z_to_y_main.enable_magnitudes = [False, True]
cfg.z_to_y_main.spectral_flags_scale_shift = [True, False]
cfg.z_to_y_main.spectral_flags_magnitudes = [False, True]
cfg.z_to_y_main.spectral_flags_hidden_layers = [False, False]
cfg.z_to_y_main.activations_scale_shift = ["relu6", "dynamic_tanh"]
cfg.z_to_y_main.activations_hidden_layers = [nn.ELU, nn.LogSigmoid]

cfg.x_to_z_delay.scale_zeroing_threshold = 1e-4
cfg.x_to_z_deri.scale_zeroing_threshold = 1e-4
cfg.z_to_x_main.scale_zeroing_threshold = 1e-4
cfg.z_push_to_z.scale_zeroing_threshold = 1e-4
cfg.z_to_y_main.scale_zeroing_threshold = 1e-4
exp = execute_model_evaluation('rossler', cfg, data_mgr, scale=False)

Shape of training data: torch.Size([36400, 3])
Shape of validation data: torch.Size([5200, 3])
Shape of testing data: torch.Size([10400, 3])
global_std.shape: torch.Size([3])
Global Std for rossler: tensor([5.0964, 4.7939, 2.8266], device='cuda:0')
Train set sample shapes: torch.Size([96, 3]), torch.Size([96, 3])
Validation set sample shapes: torch.Size([96, 3]), torch.Size([96, 3])
Test set data shapes: torch.Size([10400, 3]), torch.Size([10400, 3])
Number of batches in train_loader: 283
Batch 0: Data shape torch.Size([128, 96, 3]), Target shape torch.Size([128, 96, 3])

Data Preparation: rossler
Sequence Length: 96
Prediction Length: 96
Batch Size: 128
Scaling: No
Train Split: 0.7
Val Split: 0.8
Training Batches: 283
Validation Batches: 40
Test Batches: 80

 Running experiment with seed 1955 (1/3)==================================================

Epoch [1/50], Train Losses: mse: 2.3404, mae: 0.5379, huber: 0.3439, swd: 1.9649, ept: 91.2539
Epoch [1/50], Val Losses: mse: 0.7724, mae:

##### do not use ablations: only threshold z_to_z and z_to_y

In [14]:
from monotonic import DynamicTanh
import torch.nn as nn

importlib.reload(monotonic)
importlib.reload(train_config) 
cfg = train_config.FlatACLConfig(  # original householder 
    seq_len=96,
    pred_len=96,
    channels=data_mgr.datasets['rossler']['channels'],# data_mgr.channels,              # ← number of features in your data
    batch_size=128,
    learning_rate=9e-4, 
    seeds=[1955, 7, 20],  
    epochs=50, 
    dim_hidden=128,
    dim_augment=128, 
    ablate_no_koopman=False,
    use_complex_eigenvalues=True,
    second_delay_use_shift=True,
    ablate_rotate_back_Koopman=True, 
    ablate_shift_inside_scale=False,
    householder_reflects_latent = 2,
    householder_reflects_data = 4,
    mixing_strategy='delay_only', 
    loss_backward_weights = [0.0, 0.0, 1.0, 0.0, 0.0],
    loss_validate_weights = [0.0, 0.0, 1.0, 0.0, 0.0],
    ablate_deterministic_y0=False, 
)
cfg.x_to_z_delay.enable_magnitudes = [False, True]
cfg.x_to_z_delay.spectral_flags_scale_shift = [True, False]
cfg.x_to_z_delay.spectral_flags_magnitudes = [False, True]
cfg.x_to_z_delay.spectral_flags_hidden_layers = [False, False]
cfg.x_to_z_delay.activations_scale_shift = ["relu6", "dynamic_tanh"]
cfg.x_to_z_delay.activations_hidden_layers = [nn.ELU, nn.LogSigmoid]

cfg.x_to_z_deri.enable_magnitudes = [False, True]
cfg.x_to_z_deri.spectral_flags_scale_shift = [True, False]
cfg.x_to_z_deri.spectral_flags_magnitudes = [False, True]
cfg.x_to_z_deri.spectral_flags_hidden_layers = [False, False]
cfg.x_to_z_deri.activations_scale_shift = ["relu6", "dynamic_tanh"]
cfg.x_to_z_deri.activations_hidden_layers = [nn.ELU, nn.LogSigmoid]

cfg.z_to_x_main.enable_magnitudes = [False, True]
cfg.z_to_x_main.spectral_flags_scale_shift = [True, False]
cfg.z_to_x_main.spectral_flags_magnitudes = [False, True]
cfg.z_to_x_main.spectral_flags_hidden_layers = [False, False]
cfg.z_to_x_main.activations_scale_shift = ["relu6", "dynamic_tanh"]
cfg.z_to_x_main.activations_hidden_layers = [nn.ELU, nn.LogSigmoid]

cfg.z_push_to_z.enable_magnitudes = [False, True]
cfg.z_push_to_z.spectral_flags_scale_shift = [True, False]
cfg.z_push_to_z.spectral_flags_magnitudes = [False, True]
cfg.z_push_to_z.spectral_flags_hidden_layers = [False, False]
cfg.z_push_to_z.activations_scale_shift = ["relu6", "dynamic_tanh"]
cfg.z_push_to_z.activations_hidden_layers = [nn.ELU, nn.LogSigmoid]

cfg.z_to_y_main.enable_magnitudes = [False, True]
cfg.z_to_y_main.spectral_flags_scale_shift = [True, False]
cfg.z_to_y_main.spectral_flags_magnitudes = [False, True]
cfg.z_to_y_main.spectral_flags_hidden_layers = [False, False]
cfg.z_to_y_main.activations_scale_shift = ["relu6", "dynamic_tanh"]
cfg.z_to_y_main.activations_hidden_layers = [nn.ELU, nn.LogSigmoid]

cfg.x_to_z_delay.scale_zeroing_threshold = 0e-4
cfg.x_to_z_deri.scale_zeroing_threshold = 0e-4
cfg.z_to_x_main.scale_zeroing_threshold = 0e-4
cfg.z_push_to_z.scale_zeroing_threshold = 1e-4
cfg.z_to_y_main.scale_zeroing_threshold = 1e-4
exp = execute_model_evaluation('rossler', cfg, data_mgr, scale=False)

Shape of training data: torch.Size([36400, 3])
Shape of validation data: torch.Size([5200, 3])
Shape of testing data: torch.Size([10400, 3])
global_std.shape: torch.Size([3])
Global Std for rossler: tensor([5.0964, 4.7939, 2.8266], device='cuda:0')
Train set sample shapes: torch.Size([96, 3]), torch.Size([96, 3])
Validation set sample shapes: torch.Size([96, 3]), torch.Size([96, 3])
Test set data shapes: torch.Size([10400, 3]), torch.Size([10400, 3])
Number of batches in train_loader: 283
Batch 0: Data shape torch.Size([128, 96, 3]), Target shape torch.Size([128, 96, 3])

Data Preparation: rossler
Sequence Length: 96
Prediction Length: 96
Batch Size: 128
Scaling: No
Train Split: 0.7
Val Split: 0.8
Training Batches: 283
Validation Batches: 40
Test Batches: 80

 Running experiment with seed 1955 (1/3)==================================================

Epoch [1/50], Train Losses: mse: 2.3531, mae: 0.5433, huber: 0.3464, swd: 1.9766, ept: 91.2247
Epoch [1/50], Val Losses: mse: 0.9892, mae:

##### do not use: huber +0.1SWD

In [3]:
from monotonic import DynamicTanh
import torch.nn as nn

importlib.reload(monotonic)
importlib.reload(train_config) 
cfg = train_config.FlatACLConfig(  # original householder 
    seq_len=96,
    pred_len=96,
    channels=data_mgr.datasets['rossler']['channels'],# data_mgr.channels,              # ← number of features in your data
    batch_size=128,
    learning_rate=9e-4, 
    seeds=[1955, 7, 20],  
    epochs=50, 
    dim_hidden=128,
    dim_augment=128, 
    ablate_no_koopman=False,
    use_complex_eigenvalues=True,
    second_delay_use_shift=True,
    ablate_rotate_back_Koopman=True, 
    ablate_shift_inside_scale=False,
    householder_reflects_latent = 2,
    householder_reflects_data = 4,
    mixing_strategy='delay_only', 
    loss_backward_weights = [0.0, 0.0, 1.0, 0.1, 0.0],
    loss_validate_weights = [0.0, 0.0, 1.0, 0.1, 0.0],
    ablate_deterministic_y0=False, 
)
cfg.x_to_z_delay.enable_magnitudes = [False, True]
cfg.x_to_z_delay.spectral_flags_scale_shift = [True, False]
cfg.x_to_z_delay.spectral_flags_magnitudes = [False, True]
cfg.x_to_z_delay.spectral_flags_hidden_layers = [False, False]
cfg.x_to_z_delay.activations_scale_shift = ["relu6", "dynamic_tanh"]
cfg.x_to_z_delay.activations_hidden_layers = [nn.ELU, nn.LogSigmoid]

cfg.x_to_z_deri.enable_magnitudes = [False, True]
cfg.x_to_z_deri.spectral_flags_scale_shift = [True, False]
cfg.x_to_z_deri.spectral_flags_magnitudes = [False, True]
cfg.x_to_z_deri.spectral_flags_hidden_layers = [False, False]
cfg.x_to_z_deri.activations_scale_shift = ["relu6", "dynamic_tanh"]
cfg.x_to_z_deri.activations_hidden_layers = [nn.ELU, nn.LogSigmoid]

cfg.z_to_x_main.enable_magnitudes = [False, True]
cfg.z_to_x_main.spectral_flags_scale_shift = [True, False]
cfg.z_to_x_main.spectral_flags_magnitudes = [False, True]
cfg.z_to_x_main.spectral_flags_hidden_layers = [False, False]
cfg.z_to_x_main.activations_scale_shift = ["relu6", "dynamic_tanh"]
cfg.z_to_x_main.activations_hidden_layers = [nn.ELU, nn.LogSigmoid]

cfg.z_push_to_z.enable_magnitudes = [False, True]
cfg.z_push_to_z.spectral_flags_scale_shift = [True, False]
cfg.z_push_to_z.spectral_flags_magnitudes = [False, True]
cfg.z_push_to_z.spectral_flags_hidden_layers = [False, False]
cfg.z_push_to_z.activations_scale_shift = ["relu6", "dynamic_tanh"]
cfg.z_push_to_z.activations_hidden_layers = [nn.ELU, nn.LogSigmoid]

cfg.z_to_y_main.enable_magnitudes = [False, True]
cfg.z_to_y_main.spectral_flags_scale_shift = [True, False]
cfg.z_to_y_main.spectral_flags_magnitudes = [False, True]
cfg.z_to_y_main.spectral_flags_hidden_layers = [False, False]
cfg.z_to_y_main.activations_scale_shift = ["relu6", "dynamic_tanh"]
cfg.z_to_y_main.activations_hidden_layers = [nn.ELU, nn.LogSigmoid]
exp = execute_model_evaluation('rossler', cfg, data_mgr, scale=False)

Shape of training data: torch.Size([36400, 3])
Shape of validation data: torch.Size([5200, 3])
Shape of testing data: torch.Size([10400, 3])
global_std.shape: torch.Size([3])
Global Std for rossler: tensor([5.0964, 4.7939, 2.8266], device='cuda:0')
Train set sample shapes: torch.Size([96, 3]), torch.Size([96, 3])
Validation set sample shapes: torch.Size([96, 3]), torch.Size([96, 3])
Test set data shapes: torch.Size([10400, 3]), torch.Size([10400, 3])
Number of batches in train_loader: 283
Batch 0: Data shape torch.Size([128, 96, 3]), Target shape torch.Size([128, 96, 3])

Data Preparation: rossler
Sequence Length: 96
Prediction Length: 96
Batch Size: 128
Scaling: No
Train Split: 0.7
Val Split: 0.8
Training Batches: 283
Validation Batches: 40
Test Batches: 80

 Running experiment with seed 1955 (1/3)==================================================

Epoch [1/50], Train Losses: mse: 2.3605, mae: 0.5891, huber: 0.3722, swd: 1.5033, ept: 87.3599
Epoch [1/50], Val Losses: mse: 0.2558, mae:

#### pred=196

##### huber

In [2]:
importlib.reload(monotonic)
importlib.reload(train_config)

cfg = train_config.FlatACLConfig( 
    seq_len=96,
    pred_len=196,
    channels=data_mgr.datasets['rossler']['channels'],# data_mgr.channels,              # ← number of features in your data
    batch_size=128,
    learning_rate=9e-4, 
    seeds=[1955, 7, 20],  
    epochs=50, 
    dim_hidden=128,
    dim_augment=128,
    ablate_no_koopman=False,
    use_complex_eigenvalues=True,
    second_delay_use_shift=True,
    ablate_rotate_back_Koopman=False, 
    ablate_shift_inside_scale=False,
)
cfg.x_to_z_delay.enable_magnitudes_scale_shift = [False, True]
cfg.x_to_z_deri.enable_magnitudes_scale_shift = [False, True]
cfg.z_to_x_main.enable_magnitudes_scale_shift = [False, True]
cfg.z_to_y_main.enable_magnitudes_scale_shift = [False, True]
exp = execute_model_evaluation('rossler', cfg, data_mgr, scale=False)

Shape of training data: torch.Size([36400, 3])
Shape of validation data: torch.Size([5200, 3])
Shape of testing data: torch.Size([10400, 3])
global_std.shape: torch.Size([3])
Global Std for rossler: tensor([5.0964, 4.7939, 2.8266], device='cuda:0')
Train set sample shapes: torch.Size([96, 3]), torch.Size([196, 3])
Validation set sample shapes: torch.Size([96, 3]), torch.Size([196, 3])
Test set data shapes: torch.Size([10400, 3]), torch.Size([10400, 3])
Number of batches in train_loader: 283
Batch 0: Data shape torch.Size([128, 96, 3]), Target shape torch.Size([128, 196, 3])

Data Preparation: rossler
Sequence Length: 96
Prediction Length: 196
Batch Size: 128
Scaling: No
Train Split: 0.7
Val Split: 0.8
Training Batches: 283
Validation Batches: 39
Test Batches: 79

 Running experiment with seed 1955 (1/3)==================================================

Epoch [1/50], Train Losses: mse: 3.0439, mae: 0.6390, huber: 0.4131, swd: 2.7218, ept: 182.6913
Epoch [1/50], Val Losses: mse: 1.2717,

#### pred=336

##### huber

In [8]:
from monotonic import DynamicTanh
import torch.nn as nn

importlib.reload(monotonic)
importlib.reload(train_config) 
cfg = train_config.FlatACLConfig(  # original householder 
    seq_len=96,
    pred_len=336,
    channels=data_mgr.datasets['rossler']['channels'],# data_mgr.channels,              # ← number of features in your data
    batch_size=128,
    learning_rate=9e-4, 
    seeds=[1955, 7, 20],  
    epochs=50, 
    dim_hidden=128,
    dim_augment=128, 
    ablate_no_koopman=False,
    use_complex_eigenvalues=True,
    second_delay_use_shift=True,
    ablate_rotate_back_Koopman=True, 
    ablate_shift_inside_scale=False,
    householder_reflects_latent = 2,
    householder_reflects_data = 4,
    mixing_strategy='delay_only', 
    loss_backward_weights = [0.0, 0.0, 1.0, 0.0, 0.0],
    loss_validate_weights = [0.0, 0.0, 1.0, 0.0, 0.0],
    ablate_deterministic_y0=False, 
)
cfg.x_to_z_delay.enable_magnitudes = [False, True]
cfg.x_to_z_delay.spectral_flags_scale_shift = [True, False]
cfg.x_to_z_delay.spectral_flags_magnitudes = [False, True]
cfg.x_to_z_delay.spectral_flags_hidden_layers = [False, False]
cfg.x_to_z_delay.activations_scale_shift = ["relu6", "dynamic_tanh"]
cfg.x_to_z_delay.activations_hidden_layers = [nn.ELU, nn.LogSigmoid]

cfg.x_to_z_deri.enable_magnitudes = [False, True]
cfg.x_to_z_deri.spectral_flags_scale_shift = [True, False]
cfg.x_to_z_deri.spectral_flags_magnitudes = [False, True]
cfg.x_to_z_deri.spectral_flags_hidden_layers = [False, False]
cfg.x_to_z_deri.activations_scale_shift = ["relu6", "dynamic_tanh"]
cfg.x_to_z_deri.activations_hidden_layers = [nn.ELU, nn.LogSigmoid]

cfg.z_to_x_main.enable_magnitudes = [False, True]
cfg.z_to_x_main.spectral_flags_scale_shift = [True, False]
cfg.z_to_x_main.spectral_flags_magnitudes = [False, True]
cfg.z_to_x_main.spectral_flags_hidden_layers = [False, False]
cfg.z_to_x_main.activations_scale_shift = ["relu6", "dynamic_tanh"]
cfg.z_to_x_main.activations_hidden_layers = [nn.ELU, nn.LogSigmoid]

cfg.z_push_to_z.enable_magnitudes = [False, True]
cfg.z_push_to_z.spectral_flags_scale_shift = [True, False]
cfg.z_push_to_z.spectral_flags_magnitudes = [False, True]
cfg.z_push_to_z.spectral_flags_hidden_layers = [False, False]
cfg.z_push_to_z.activations_scale_shift = ["relu6", "dynamic_tanh"]
cfg.z_push_to_z.activations_hidden_layers = [nn.ELU, nn.LogSigmoid]

cfg.z_to_y_main.enable_magnitudes = [False, True]
cfg.z_to_y_main.spectral_flags_scale_shift = [True, False]
cfg.z_to_y_main.spectral_flags_magnitudes = [False, True]
cfg.z_to_y_main.spectral_flags_hidden_layers = [False, False]
cfg.z_to_y_main.activations_scale_shift = ["relu6", "dynamic_tanh"]
cfg.z_to_y_main.activations_hidden_layers = [nn.ELU, nn.LogSigmoid]
exp = execute_model_evaluation('rossler', cfg, data_mgr, scale=False)

Shape of training data: torch.Size([36400, 3])
Shape of validation data: torch.Size([5200, 3])
Shape of testing data: torch.Size([10400, 3])
global_std.shape: torch.Size([3])
Global Std for rossler: tensor([5.0964, 4.7939, 2.8266], device='cuda:0')
Train set sample shapes: torch.Size([96, 3]), torch.Size([336, 3])
Validation set sample shapes: torch.Size([96, 3]), torch.Size([336, 3])
Test set data shapes: torch.Size([10400, 3]), torch.Size([10400, 3])
Number of batches in train_loader: 282
Batch 0: Data shape torch.Size([128, 96, 3]), Target shape torch.Size([128, 336, 3])

Data Preparation: rossler
Sequence Length: 96
Prediction Length: 336
Batch Size: 128
Scaling: No
Train Split: 0.7
Val Split: 0.8
Training Batches: 282
Validation Batches: 38
Test Batches: 78

 Running experiment with seed 1955 (1/3)==================================================

Epoch [1/50], Train Losses: mse: 3.8776, mae: 0.7598, huber: 0.5186, swd: 3.0869, ept: 295.3539
Epoch [1/50], Val Losses: mse: 3.5122,

##### do not use ablation: rotation (8,4)

In [3]:
importlib.reload(monotonic)
importlib.reload(train_config)

cfg = train_config.FlatACLConfig( 
    seq_len=96,
    pred_len=336,
    channels=data_mgr.datasets['rossler']['channels'],# data_mgr.channels,              # ← number of features in your data
    batch_size=128,
    learning_rate=9e-4, 
    seeds=[1955, 7, 20],  
    epochs=50, 
    dim_hidden=128,
    dim_augment=128,
    ablate_no_koopman=False,
    use_complex_eigenvalues=True,
    second_delay_use_shift=True,
    ablate_rotate_back_Koopman=False, 
    ablate_shift_inside_scale=False,
)
cfg.x_to_z_delay.enable_magnitudes_scale_shift = [False, True]
cfg.x_to_z_deri.enable_magnitudes_scale_shift = [False, True]
cfg.z_to_x_main.enable_magnitudes_scale_shift = [False, True]
cfg.z_to_y_main.enable_magnitudes_scale_shift = [False, True]
exp = execute_model_evaluation('rossler', cfg, data_mgr, scale=False)

Shape of training data: torch.Size([36400, 3])
Shape of validation data: torch.Size([5200, 3])
Shape of testing data: torch.Size([10400, 3])
Train set sample shapes: torch.Size([96, 3]), torch.Size([336, 3])
Validation set sample shapes: torch.Size([96, 3]), torch.Size([336, 3])
Test set data shapes: torch.Size([10400, 3]), torch.Size([10400, 3])
Number of batches in train_loader: 282
Batch 0: Data shape torch.Size([128, 96, 3]), Target shape torch.Size([128, 336, 3])

Data Preparation: rossler
Sequence Length: 96
Prediction Length: 336
Batch Size: 128
Scaling: No
Train Split: 0.7
Val Split: 0.8
Training Batches: 282
Validation Batches: 38
Test Batches: 78

 Running experiment with seed 1955 (1/3)==================================================

Epoch [1/50], Train Losses: mse: 4.0875, mae: 0.7933, huber: 0.5457, swd: 3.3423, target_std: 3.9412
Epoch [1/50], Val Losses: mse: 4.5950, mae: 0.8581, huber: 0.5837, swd: 2.9817, target_std: 3.9670
Epoch [1/50], Test Losses: mse: 4.3068, ma

#### pred=720

##### huber

In [3]:
importlib.reload(monotonic)
importlib.reload(train_config)

cfg = train_config.FlatACLConfig( 
    seq_len=96,
    pred_len=720,
    channels=data_mgr.datasets['rossler']['channels'],# data_mgr.channels,              # ← number of features in your data
    batch_size=128,
    learning_rate=9e-4, 
    seeds=[1955, 7, 20],  
    epochs=50, 
    dim_hidden=128,
    dim_augment=128,
    ablate_no_koopman=False,
    use_complex_eigenvalues=True,
    second_delay_use_shift=True,
    ablate_rotate_back_Koopman=False, 
    ablate_shift_inside_scale=False,
)
cfg.x_to_z_delay.enable_magnitudes_scale_shift = [False, True]
cfg.x_to_z_deri.enable_magnitudes_scale_shift = [False, True]
cfg.z_to_x_main.enable_magnitudes_scale_shift = [False, True]
cfg.z_to_y_main.enable_magnitudes_scale_shift = [False, True]
exp = execute_model_evaluation('rossler', cfg, data_mgr, scale=False)

Shape of training data: torch.Size([36400, 3])
Shape of validation data: torch.Size([5200, 3])
Shape of testing data: torch.Size([10400, 3])
global_std.shape: torch.Size([3])
Global Std for rossler: tensor([5.0964, 4.7939, 2.8266], device='cuda:0')
Train set sample shapes: torch.Size([96, 3]), torch.Size([720, 3])
Validation set sample shapes: torch.Size([96, 3]), torch.Size([720, 3])
Test set data shapes: torch.Size([10400, 3]), torch.Size([10400, 3])
Number of batches in train_loader: 279
Batch 0: Data shape torch.Size([128, 96, 3]), Target shape torch.Size([128, 720, 3])

Data Preparation: rossler
Sequence Length: 96
Prediction Length: 720
Batch Size: 128
Scaling: No
Train Split: 0.7
Val Split: 0.8
Training Batches: 279
Validation Batches: 35
Test Batches: 75

 Running experiment with seed 1955 (1/3)==================================================

Epoch [1/50], Train Losses: mse: 4.0925, mae: 0.8460, huber: 0.5809, swd: 3.0309, ept: 576.6307
Epoch [1/50], Val Losses: mse: 4.6120,

### Timemixer

#### pred=96

##### huber

In [2]:
utils.reload_modules([utils])
cfg = train_config.FlatTimeMixerConfig(
    seq_len=96,
    pred_len=96,
    channels=data_mgr.datasets['rossler']['channels'],
    enc_in=data_mgr.datasets['rossler']['channels'],
    dec_in=data_mgr.datasets['rossler']['channels'],
    c_out=data_mgr.datasets['rossler']['channels'],
    batch_size=128,
    learning_rate=9e-4,
    seeds=[1955, 7, 20],
    epochs=50, 
)
exp_mixer_96_96 = execute_model_evaluation('rossler', cfg, data_mgr, scale=False)


Reloading modules...
  Reloaded: utils
Module reload complete.
Shape of training data: torch.Size([36400, 3])
Shape of validation data: torch.Size([5200, 3])
Shape of testing data: torch.Size([10400, 3])
global_std.shape: torch.Size([3])
Global Std for rossler: tensor([5.0964, 4.7939, 2.8266], device='cuda:0')
Train set sample shapes: torch.Size([96, 3]), torch.Size([96, 3])
Validation set sample shapes: torch.Size([96, 3]), torch.Size([96, 3])
Test set data shapes: torch.Size([10400, 3]), torch.Size([10400, 3])
Number of batches in train_loader: 283
Batch 0: Data shape torch.Size([128, 96, 3]), Target shape torch.Size([128, 96, 3])

Data Preparation: rossler
Sequence Length: 96
Prediction Length: 96
Batch Size: 128
Scaling: No
Train Split: 0.7
Val Split: 0.8
Training Batches: 283
Validation Batches: 40
Test Batches: 80

 Running experiment with seed 1955 (1/3)==================================================

Epoch [1/50], Train Losses: mse: 7.0772, mae: 1.1023, huber: 0.8271, swd: 4

#### pred=196

##### huber

In [15]:
utils.reload_modules([utils])
cfg = train_config.FlatTimeMixerConfig(
    seq_len=96,
    pred_len=196,
    channels=data_mgr.datasets['rossler']['channels'],
    enc_in=data_mgr.datasets['rossler']['channels'],
    dec_in=data_mgr.datasets['rossler']['channels'],
    c_out=data_mgr.datasets['rossler']['channels'],
    batch_size=128,
    learning_rate=9e-4,
    seeds=[1955, 7, 20],
    epochs=50, 
)
exp = execute_model_evaluation('rossler', cfg, data_mgr, scale=False)


Reloading modules...
  Reloaded: utils
Module reload complete.
Shape of training data: torch.Size([36400, 3])
Shape of validation data: torch.Size([5200, 3])
Shape of testing data: torch.Size([10400, 3])
global_std.shape: torch.Size([3])
Global Std for rossler: tensor([5.0964, 4.7939, 2.8266], device='cuda:0')
Train set sample shapes: torch.Size([96, 3]), torch.Size([196, 3])
Validation set sample shapes: torch.Size([96, 3]), torch.Size([196, 3])
Test set data shapes: torch.Size([10400, 3]), torch.Size([10400, 3])
Number of batches in train_loader: 283
Batch 0: Data shape torch.Size([128, 96, 3]), Target shape torch.Size([128, 196, 3])

Data Preparation: rossler
Sequence Length: 96
Prediction Length: 196
Batch Size: 128
Scaling: No
Train Split: 0.7
Val Split: 0.8
Training Batches: 283
Validation Batches: 39
Test Batches: 79

 Running experiment with seed 1955 (1/3)==================================================

Epoch [1/50], Train Losses: mse: 16.2527, mae: 1.9405, huber: 1.6176, s

#### pred=336

##### huber

In [9]:
utils.reload_modules([utils])
cfg = train_config.FlatTimeMixerConfig(
    seq_len=96,
    pred_len=336,
    channels=data_mgr.datasets['rossler']['channels'],
    enc_in=data_mgr.datasets['rossler']['channels'],
    dec_in=data_mgr.datasets['rossler']['channels'],
    c_out=data_mgr.datasets['rossler']['channels'],
    batch_size=128,
    learning_rate=9e-4,
    seeds=[1955, 7, 20],
    epochs=50, 
)
exp = execute_model_evaluation('rossler', cfg, data_mgr, scale=False)


Reloading modules...
  Reloaded: utils
Module reload complete.
Shape of training data: torch.Size([36400, 3])
Shape of validation data: torch.Size([5200, 3])
Shape of testing data: torch.Size([10400, 3])
global_std.shape: torch.Size([3])
Global Std for rossler: tensor([5.0964, 4.7939, 2.8266], device='cuda:0')
Train set sample shapes: torch.Size([96, 3]), torch.Size([336, 3])
Validation set sample shapes: torch.Size([96, 3]), torch.Size([336, 3])
Test set data shapes: torch.Size([10400, 3]), torch.Size([10400, 3])
Number of batches in train_loader: 282
Batch 0: Data shape torch.Size([128, 96, 3]), Target shape torch.Size([128, 336, 3])

Data Preparation: rossler
Sequence Length: 96
Prediction Length: 336
Batch Size: 128
Scaling: No
Train Split: 0.7
Val Split: 0.8
Training Batches: 282
Validation Batches: 38
Test Batches: 78

 Running experiment with seed 1955 (1/3)==================================================

Epoch [1/50], Train Losses: mse: 26.1457, mae: 2.7262, huber: 2.3707, s

In [18]:
utils.reload_modules([utils])
cfg = train_config.FlatTimeMixerConfig(
    seq_len=96,
    pred_len=336,
    channels=data_mgr.datasets['rossler']['channels'],
    enc_in=data_mgr.datasets['rossler']['channels'],
    dec_in=data_mgr.datasets['rossler']['channels'],
    c_out=data_mgr.datasets['rossler']['channels'],
    batch_size=128,
    learning_rate=9e-4,
    seeds=[1955, 7, 20],
    epochs=50, 
)
exp = execute_model_evaluation('rossler', cfg, data_mgr, scale=False)


Reloading modules...
  Reloaded: utils
Module reload complete.
Shape of training data: torch.Size([36400, 3])
Shape of validation data: torch.Size([5200, 3])
Shape of testing data: torch.Size([10400, 3])
Train set sample shapes: torch.Size([96, 3]), torch.Size([336, 3])
Validation set sample shapes: torch.Size([96, 3]), torch.Size([336, 3])
Test set data shapes: torch.Size([10400, 3]), torch.Size([10400, 3])
Number of batches in train_loader: 282
Batch 0: Data shape torch.Size([128, 96, 3]), Target shape torch.Size([128, 336, 3])

Data Preparation: rossler
Sequence Length: 96
Prediction Length: 336
Batch Size: 128
Scaling: No
Train Split: 0.7
Val Split: 0.8
Training Batches: 282
Validation Batches: 38
Test Batches: 78

 Running experiment with seed 1955 (1/3)==================================================

Epoch [1/50], Train Losses: mse: 26.1457, mae: 2.7262, huber: 2.3707, swd: 13.9791, target_std: 3.9435
Epoch [1/50], Val Losses: mse: 19.2632, mae: 2.3370, huber: 1.9792, swd: 13.

#### pred=720

##### huber

In [16]:
utils.reload_modules([utils])
cfg = train_config.FlatTimeMixerConfig(
    seq_len=96,
    pred_len=720,
    channels=data_mgr.datasets['rossler']['channels'],
    enc_in=data_mgr.datasets['rossler']['channels'],
    dec_in=data_mgr.datasets['rossler']['channels'],
    c_out=data_mgr.datasets['rossler']['channels'],
    batch_size=128,
    learning_rate=9e-4,
    seeds=[1955, 7, 20],
    epochs=50, 
)
exp = execute_model_evaluation('rossler', cfg, data_mgr, scale=False)


Reloading modules...
  Reloaded: utils
Module reload complete.
Shape of training data: torch.Size([36400, 3])
Shape of validation data: torch.Size([5200, 3])
Shape of testing data: torch.Size([10400, 3])
global_std.shape: torch.Size([3])
Global Std for rossler: tensor([5.0964, 4.7939, 2.8266], device='cuda:0')
Train set sample shapes: torch.Size([96, 3]), torch.Size([720, 3])
Validation set sample shapes: torch.Size([96, 3]), torch.Size([720, 3])
Test set data shapes: torch.Size([10400, 3]), torch.Size([10400, 3])
Number of batches in train_loader: 279
Batch 0: Data shape torch.Size([128, 96, 3]), Target shape torch.Size([128, 720, 3])

Data Preparation: rossler
Sequence Length: 96
Prediction Length: 720
Batch Size: 128
Scaling: No
Train Split: 0.7
Val Split: 0.8
Training Batches: 279
Validation Batches: 35
Test Batches: 75

 Running experiment with seed 1955 (1/3)==================================================

Epoch [1/50], Train Losses: mse: 20.5072, mae: 2.6311, huber: 2.2693, s

### PatchTST

#### pred=96

##### huber

In [17]:
utils.reload_modules([utils])
cfg = train_config.FlatPatchTSTConfig(
    seq_len=96,
    pred_len=96,
    channels=data_mgr.datasets['rossler']['channels'],
    enc_in=data_mgr.datasets['rossler']['channels'],
    dec_in=data_mgr.datasets['rossler']['channels'],
    c_out=data_mgr.datasets['rossler']['channels'],
    batch_size=128,
    learning_rate=9e-4,
    seeds=[1955, 7, 20],
    epochs=50,
    task_name='long_term_forecast',
    factor=3,
)
exp = execute_model_evaluation('rossler', cfg, data_mgr, scale=False)

Reloading modules...
  Reloaded: utils
Module reload complete.
Shape of training data: torch.Size([36400, 3])
Shape of validation data: torch.Size([5200, 3])
Shape of testing data: torch.Size([10400, 3])
global_std.shape: torch.Size([3])
Global Std for rossler: tensor([5.0964, 4.7939, 2.8266], device='cuda:0')
Train set sample shapes: torch.Size([96, 3]), torch.Size([96, 3])
Validation set sample shapes: torch.Size([96, 3]), torch.Size([96, 3])
Test set data shapes: torch.Size([10400, 3]), torch.Size([10400, 3])
Number of batches in train_loader: 283
Batch 0: Data shape torch.Size([128, 96, 3]), Target shape torch.Size([128, 96, 3])

Data Preparation: rossler
Sequence Length: 96
Prediction Length: 96
Batch Size: 128
Scaling: No
Train Split: 0.7
Val Split: 0.8
Training Batches: 283
Validation Batches: 40
Test Batches: 80

 Running experiment with seed 1955 (1/3)==================================================

Epoch [1/50], Train Losses: mse: 9.4284, mae: 1.2200, huber: 0.9393, swd: 5

#### pred=196

##### huber

In [18]:
utils.reload_modules([utils])
cfg = train_config.FlatPatchTSTConfig(
    seq_len=96,
    pred_len=196,
    channels=data_mgr.datasets['rossler']['channels'],
    enc_in=data_mgr.datasets['rossler']['channels'],
    dec_in=data_mgr.datasets['rossler']['channels'],
    c_out=data_mgr.datasets['rossler']['channels'],
    batch_size=128,
    learning_rate=9e-4,
    seeds=[1955, 7, 20],
    epochs=50,
    task_name='long_term_forecast',
    factor=3,
)
exp = execute_model_evaluation('rossler', cfg, data_mgr, scale=False)

Reloading modules...
  Reloaded: utils
Module reload complete.
Shape of training data: torch.Size([36400, 3])
Shape of validation data: torch.Size([5200, 3])
Shape of testing data: torch.Size([10400, 3])
global_std.shape: torch.Size([3])
Global Std for rossler: tensor([5.0964, 4.7939, 2.8266], device='cuda:0')
Train set sample shapes: torch.Size([96, 3]), torch.Size([196, 3])
Validation set sample shapes: torch.Size([96, 3]), torch.Size([196, 3])
Test set data shapes: torch.Size([10400, 3]), torch.Size([10400, 3])
Number of batches in train_loader: 283
Batch 0: Data shape torch.Size([128, 96, 3]), Target shape torch.Size([128, 196, 3])

Data Preparation: rossler
Sequence Length: 96
Prediction Length: 196
Batch Size: 128
Scaling: No
Train Split: 0.7
Val Split: 0.8
Training Batches: 283
Validation Batches: 39
Test Batches: 79

 Running experiment with seed 1955 (1/3)==================================================

Epoch [1/50], Train Losses: mse: 18.7758, mae: 2.2311, huber: 1.8947, s

#### pred=336

##### huber

In [19]:
utils.reload_modules([utils])
cfg = train_config.FlatPatchTSTConfig(
    seq_len=96,
    pred_len=336,
    channels=data_mgr.datasets['rossler']['channels'],
    enc_in=data_mgr.datasets['rossler']['channels'],
    dec_in=data_mgr.datasets['rossler']['channels'],
    c_out=data_mgr.datasets['rossler']['channels'],
    batch_size=128,
    learning_rate=9e-4,
    seeds=[1955, 7, 20],
    epochs=50,
    task_name='long_term_forecast',
    factor=3,
)
exp = execute_model_evaluation('rossler', cfg, data_mgr, scale=False)

Reloading modules...
  Reloaded: utils
Module reload complete.
Shape of training data: torch.Size([36400, 3])
Shape of validation data: torch.Size([5200, 3])
Shape of testing data: torch.Size([10400, 3])
global_std.shape: torch.Size([3])
Global Std for rossler: tensor([5.0964, 4.7939, 2.8266], device='cuda:0')
Train set sample shapes: torch.Size([96, 3]), torch.Size([336, 3])
Validation set sample shapes: torch.Size([96, 3]), torch.Size([336, 3])
Test set data shapes: torch.Size([10400, 3]), torch.Size([10400, 3])
Number of batches in train_loader: 282
Batch 0: Data shape torch.Size([128, 96, 3]), Target shape torch.Size([128, 336, 3])

Data Preparation: rossler
Sequence Length: 96
Prediction Length: 336
Batch Size: 128
Scaling: No
Train Split: 0.7
Val Split: 0.8
Training Batches: 282
Validation Batches: 38
Test Batches: 78

 Running experiment with seed 1955 (1/3)==================================================

Epoch [1/50], Train Losses: mse: 26.9142, mae: 2.9954, huber: 2.6311, s

#### pred=720

##### huber

In [20]:
utils.reload_modules([utils])
cfg = train_config.FlatPatchTSTConfig(
    seq_len=96,
    pred_len=720,
    channels=data_mgr.datasets['rossler']['channels'],
    enc_in=data_mgr.datasets['rossler']['channels'],
    dec_in=data_mgr.datasets['rossler']['channels'],
    c_out=data_mgr.datasets['rossler']['channels'],
    batch_size=128,
    learning_rate=9e-4,
    seeds=[1955, 7, 20],
    epochs=50,
    task_name='long_term_forecast',
    factor=3,
)
exp = execute_model_evaluation('rossler', cfg, data_mgr, scale=False)

Reloading modules...
  Reloaded: utils
Module reload complete.
Shape of training data: torch.Size([36400, 3])
Shape of validation data: torch.Size([5200, 3])
Shape of testing data: torch.Size([10400, 3])
global_std.shape: torch.Size([3])
Global Std for rossler: tensor([5.0964, 4.7939, 2.8266], device='cuda:0')
Train set sample shapes: torch.Size([96, 3]), torch.Size([720, 3])
Validation set sample shapes: torch.Size([96, 3]), torch.Size([720, 3])
Test set data shapes: torch.Size([10400, 3]), torch.Size([10400, 3])
Number of batches in train_loader: 279
Batch 0: Data shape torch.Size([128, 96, 3]), Target shape torch.Size([128, 720, 3])

Data Preparation: rossler
Sequence Length: 96
Prediction Length: 720
Batch Size: 128
Scaling: No
Train Split: 0.7
Val Split: 0.8
Training Batches: 279
Validation Batches: 35
Test Batches: 75

 Running experiment with seed 1955 (1/3)==================================================

Epoch [1/50], Train Losses: mse: 21.7126, mae: 2.7784, huber: 2.4121, s

### DLinear

#### pred=96

##### huber

In [21]:
importlib.reload(monotonic)
importlib.reload(train_config)
utils.reload_modules([utils])
cfg = train_config.FlatDLinearConfig(
    seq_len=96,
    pred_len=96,
    channels=data_mgr.datasets['rossler']['channels'],
    batch_size=128,
    learning_rate=9e-4,
    seeds=[1955, 7, 20],
    epochs=50, 
)
exp = execute_model_evaluation('rossler', cfg, data_mgr, scale=False)

Reloading modules...
  Reloaded: utils
Module reload complete.
Shape of training data: torch.Size([36400, 3])
Shape of validation data: torch.Size([5200, 3])
Shape of testing data: torch.Size([10400, 3])
global_std.shape: torch.Size([3])
Global Std for rossler: tensor([5.0964, 4.7939, 2.8266], device='cuda:0')
Train set sample shapes: torch.Size([96, 3]), torch.Size([96, 3])
Validation set sample shapes: torch.Size([96, 3]), torch.Size([96, 3])
Test set data shapes: torch.Size([10400, 3]), torch.Size([10400, 3])
Number of batches in train_loader: 283
Batch 0: Data shape torch.Size([128, 96, 3]), Target shape torch.Size([128, 96, 3])

Data Preparation: rossler
Sequence Length: 96
Prediction Length: 96
Batch Size: 128
Scaling: No
Train Split: 0.7
Val Split: 0.8
Training Batches: 283
Validation Batches: 40
Test Batches: 80

 Running experiment with seed 1955 (1/3)==================================================

Epoch [1/50], Train Losses: mse: 4.2109, mae: 0.8340, huber: 0.6046, swd: 3

#### pred=196

##### huber

In [22]:
importlib.reload(monotonic)
importlib.reload(train_config)
utils.reload_modules([utils])
cfg = train_config.FlatDLinearConfig(
    seq_len=96,
    pred_len=196,
    channels=data_mgr.datasets['rossler']['channels'],
    batch_size=128,
    learning_rate=9e-4,
    seeds=[1955, 7, 20],
    epochs=50, 
)
exp = execute_model_evaluation('rossler', cfg, data_mgr, scale=False)

Reloading modules...
  Reloaded: utils
Module reload complete.
Shape of training data: torch.Size([36400, 3])
Shape of validation data: torch.Size([5200, 3])
Shape of testing data: torch.Size([10400, 3])
global_std.shape: torch.Size([3])
Global Std for rossler: tensor([5.0964, 4.7939, 2.8266], device='cuda:0')
Train set sample shapes: torch.Size([96, 3]), torch.Size([196, 3])
Validation set sample shapes: torch.Size([96, 3]), torch.Size([196, 3])
Test set data shapes: torch.Size([10400, 3]), torch.Size([10400, 3])
Number of batches in train_loader: 283
Batch 0: Data shape torch.Size([128, 96, 3]), Target shape torch.Size([128, 196, 3])

Data Preparation: rossler
Sequence Length: 96
Prediction Length: 196
Batch Size: 128
Scaling: No
Train Split: 0.7
Val Split: 0.8
Training Batches: 283
Validation Batches: 39
Test Batches: 79

 Running experiment with seed 1955 (1/3)==================================================

Epoch [1/50], Train Losses: mse: 6.4712, mae: 1.1479, huber: 0.8760, sw

#### pred=336

##### huber

In [23]:
importlib.reload(monotonic)
importlib.reload(train_config)
utils.reload_modules([utils])
cfg = train_config.FlatDLinearConfig(
    seq_len=96,
    pred_len=336,
    channels=data_mgr.datasets['rossler']['channels'],
    batch_size=128,
    learning_rate=9e-4,
    seeds=[1955, 7, 20],
    epochs=50, 
)
exp = execute_model_evaluation('rossler', cfg, data_mgr, scale=False)

Reloading modules...
  Reloaded: utils
Module reload complete.
Shape of training data: torch.Size([36400, 3])
Shape of validation data: torch.Size([5200, 3])
Shape of testing data: torch.Size([10400, 3])
global_std.shape: torch.Size([3])
Global Std for rossler: tensor([5.0964, 4.7939, 2.8266], device='cuda:0')
Train set sample shapes: torch.Size([96, 3]), torch.Size([336, 3])
Validation set sample shapes: torch.Size([96, 3]), torch.Size([336, 3])
Test set data shapes: torch.Size([10400, 3]), torch.Size([10400, 3])
Number of batches in train_loader: 282
Batch 0: Data shape torch.Size([128, 96, 3]), Target shape torch.Size([128, 336, 3])

Data Preparation: rossler
Sequence Length: 96
Prediction Length: 336
Batch Size: 128
Scaling: No
Train Split: 0.7
Val Split: 0.8
Training Batches: 282
Validation Batches: 38
Test Batches: 78

 Running experiment with seed 1955 (1/3)==================================================

Epoch [1/50], Train Losses: mse: 7.7159, mae: 1.3246, huber: 1.0267, sw

#### pred=720

##### huber

In [24]:
importlib.reload(monotonic)
importlib.reload(train_config)
utils.reload_modules([utils])
cfg = train_config.FlatDLinearConfig(
    seq_len=96,
    pred_len=720,
    channels=data_mgr.datasets['rossler']['channels'],
    batch_size=128,
    learning_rate=9e-4,
    seeds=[1955, 7, 20],
    epochs=50, 
)
exp = execute_model_evaluation('rossler', cfg, data_mgr, scale=False)

Reloading modules...
  Reloaded: utils
Module reload complete.
Shape of training data: torch.Size([36400, 3])
Shape of validation data: torch.Size([5200, 3])
Shape of testing data: torch.Size([10400, 3])
global_std.shape: torch.Size([3])
Global Std for rossler: tensor([5.0964, 4.7939, 2.8266], device='cuda:0')
Train set sample shapes: torch.Size([96, 3]), torch.Size([720, 3])
Validation set sample shapes: torch.Size([96, 3]), torch.Size([720, 3])
Test set data shapes: torch.Size([10400, 3]), torch.Size([10400, 3])
Number of batches in train_loader: 279
Batch 0: Data shape torch.Size([128, 96, 3]), Target shape torch.Size([128, 720, 3])

Data Preparation: rossler
Sequence Length: 96
Prediction Length: 720
Batch Size: 128
Scaling: No
Train Split: 0.7
Val Split: 0.8
Training Batches: 279
Validation Batches: 35
Test Batches: 75

 Running experiment with seed 1955 (1/3)==================================================

Epoch [1/50], Train Losses: mse: 9.1040, mae: 1.7161, huber: 1.3810, sw